## Daily Pivot

In [9]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/mskhelper/')
import pandas as pd, numpy as np
import logging
import datetime, pytz
import mohsenutils, data_provider, m100_preprocessing_helper
import imp 
imp.reload(mohsenutils)
imp.reload(data_provider)
imp.reload(m100_preprocessing_helper)


pub = 'logics_pub'

for metric in ['Tot_chiller', 'Tot_qpompe', 'Tot_ict', 'Tot_cdz']:
    
    
    print('__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__')
    logging.info(str('__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'+metric+'__'))
    log_file_name = 'modbus_log_raw_data_cook_'+datetime.datetime.strftime(datetime.datetime.now(pytz.timezone('Europe/Rome')), 
                                                                      '%Y-%m-%d %H:%M:%S').replace(':','-').replace(' ','_')+'.log'
    logging.basicConfig(filename=log_file_name,level=logging.INFO, format='%(levelname)s:%(asctime)s:%(lineno)d:%(message)s')


    path = os.path.join('/home/seyedkazemi/dataset/Marconi100_backup/',pub,metric,'')

    if not os.path.isdir(path+'/'+metric+'_done/'):
        os.makedirs(path+'/'+metric+'_done/')

    if not os.path.isdir(path+'/'+metric+'_pivot/'):
        os.makedirs(path+'/'+metric+'_pivot/')    

    files_list = sorted([file for file in os.listdir(path) if '.csv.gz' in file])
    for file in files_list:
        print(file)
        logging.info(str(file))
        
        data = pd.read_csv(path+file, compression='gzip', index_col='timestamp')
        
        data.loc[data['value'] == 'Unavailable', 'value'] = np.nan
        data.loc[data['value'] == 'unknown', 'value'] = np.nan
        data['value'] = data['value'].astype(float)


        
        print('Data shape: ', data.shape)
        logging.info('Data shape: '+ str(data.shape))


        data = data.pipe(m100_preprocessing_helper.remove_Unnamed, log=True).\
        pipe(m100_preprocessing_helper.unique_val, log=True).\
        pipe(m100_preprocessing_helper.pvt, index='timestamp', freq='10T', columns='name', values='value', log=True)

        data.to_csv(path+'/'+metric+'_pivot/'+file.split('.')[0]+'.csv.gz', compression='gzip')
        os.replace(path+file, path+metric+'_done/'+file)
        print(50*'#')
        logging.info(str(50*'#'))
    
    
   

__Tot_chiller__Tot_chiller__Tot_chiller__Tot_chiller__Tot_chiller__Tot_chiller__
__Tot_qpompe__Tot_qpompe__Tot_qpompe__Tot_qpompe__Tot_qpompe__Tot_qpompe__
__Tot_ict__Tot_ict__Tot_ict__Tot_ict__Tot_ict__Tot_ict__
__Tot_cdz__Tot_cdz__Tot_cdz__Tot_cdz__Tot_cdz__Tot_cdz__


## Create One Big file

In [10]:
%reset -f
import sys, os
sys.path.append('/home/seyedkazemi/codes/HPCRoomModel/mskhelper/')
import pandas as pd, numpy as np
import glob
import logging
import datetime, pytz
import mohsenutils
import imp 
import m100_preprocessing_helper
imp.reload(m100_preprocessing_helper)



pth = '/home/seyedkazemi/dataset/Marconi100_backup/'


for m in ['Tot_chiller','Tot_qpompe','Tot_cdz','Tot_ict']:
    csvdir = os.path.join(pth, 'logics_pub', m, m+'_pivot')
    m100_preprocessing_helper.concat_multiple_csvs(csvdir=csvdir,
                                                   big_csv_path_name=os.path.join(csvdir, m+'_big.csv.gz'), 
                                                   old_pivot_path=os.path.join(pth, 'logics_pub', m, m+'_pivot_old_pivot',''), 
                                                   freq='10T')

['/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_chiller/Tot_chiller_pivot/Tot_chiller_big.csv.gz']
Number of file 1
Your pivot already has been updated.
['/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_qpompe/Tot_qpompe_pivot/Tot_qpompe_big.csv.gz']
Number of file 1
Your pivot already has been updated.
['/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_cdz/Tot_cdz_pivot/Tot_cdz_big.csv.gz']
Number of file 1
Your pivot already has been updated.
['/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_ict/Tot_ict_pivot/Tot_ict_big.csv.gz']
Number of file 1
Your pivot already has been updated.


## COPY Temp DATA

In [11]:
from shutil import copyfile
copyfile('/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_chiller/Tot_chiller_pivot/Tot_chiller_big.csv.gz',
         '/home/seyedkazemi/dataset/M100_Complete/Tot_chiller_big.csv.gz')

copyfile('/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_qpompe/Tot_qpompe_pivot/Tot_qpompe_big.csv.gz',
         '/home/seyedkazemi/dataset/M100_Complete/Tot_qpompe_big.csv.gz')


copyfile('/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_cdz/Tot_cdz_pivot/Tot_cdz_big.csv.gz',
         '/home/seyedkazemi/dataset/M100_Complete/Tot_cdz_big.csv.gz')

copyfile('/home/seyedkazemi/dataset/Marconi100_backup/logics_pub/Tot_ict/Tot_ict_pivot/Tot_ict_big.csv.gz',
         '/home/seyedkazemi/dataset/M100_Complete/Tot_ict_big.csv.gz')


'/home/seyedkazemi/dataset/M100_Complete/Tot_ict_big.csv.gz'